In [11]:
import pysindy as ps

import deepSI
from deepSI import System_data
from deepSI.fit_systems import SS_encoder_general
from deepSI.fit_systems.encoders import default_encoder_net, default_state_net, default_output_net
from deepSI.fit_systems.fit_system import My_Simple_DataLoader, print_array_byte_size, Tictoctimer
import torch
from torch import nn

import numpy as np

from sklearn.preprocessing import PolynomialFeatures

from scipy.io import loadmat
import scipy.linalg as lin
import os

from utils import load_data
from data_processing import add_gaussian_noise

import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import time
from copy import deepcopy
import itertools

import re

from feature_construction import feature_library
import polynomial as p
import fourier as f

from SI_SUBNET import SS_encoder_general_eq, h_identity, e_identity, simple_Linear
from SI_metrics import plot_coeff_grid, display_equation
from run_SISUBNET import run_SISUBNET

import random
import string

from scipy.io import loadmat, savemat
import json
import pickle

In [12]:
nf = 5
gamma = np.array([1e-1])
epo_idx = 9

pruning = True
pruning_mode = "RMS"
mode = None
T_idx = None

pre_train = False if nf==1 else True
batch_size = 256
epochs = 100
no_reg_epochs = 50

threshold = 8*10**-5

In [13]:
data_set_name="sc_arrow_noisy"
x_data, u_data, y_data, eq, T, U, set_idx = load_data(set=data_set_name)
[N_arrow, N_train, N_val, N_test] = set_idx[0]

if T is None:
    T = np.eye(x_data.shape[-1])
if U is None:
    U = np.eye(u_data.shape[-1])

# data organizing
x_arrow, u_arrow = x_data[:N_arrow], u_data[:N_arrow,0]
x_train, u_train = x_data[N_arrow:N_arrow+N_train-N_val,:], u_data[N_arrow:N_arrow+N_train-N_val,0]
x_val, u_val     = x_data[N_arrow+N_train-N_val:-N_test,:], u_data[N_arrow+N_train-N_val:-N_test,0]
x_test, u_test   = x_data[-N_test:,:], u_data[-N_test:]

# system data
arrow   = System_data(u=u_arrow, y=x_arrow)
train   = System_data(u=u_train, y=x_train)
val     = System_data(u=u_val[:1000], y=x_val[:1000,:])
test    = System_data(u=u_test, y=x_test)

# function lib
functions = [p.f, f.sin, f.cos, f.sin2x, f.cos2x, f.sin3x, f.cos3x, f.sin4x, f.cos4x, f.sin5x, f.cos5x]
poly = feature_library(functions=functions, nx=2, nu=1, T=T, U=U)

# network param
nx, nu = 2, 1 # state dimension and inputs
na, nb = 0, 0
na_right = 1

f_net_kwargs = {"feature_library": poly, "u": nu, "nf": poly.feature_number()}

# output model
h_net = h_identity
h_net_kwargs = {}

# encoder model
e_net = e_identity
e_net_kwargs = {}

Data loaded from sc_arrow_noisy!


In [14]:
def convert_float_to_string(number):
    coefficient, exponent = f"{number:.1e}".split('e')
    return f"{int(float(coefficient))}em{-int(exponent)}"

In [15]:
for gi in gamma:
    set_name = "lambda_nf5_{}".format(convert_float_to_string(gi))
    print(set_name)

    run_SISUBNET(nx, na, nb, na_right, \
             f_net_kwargs, e_net, e_net_kwargs, h_net, h_net_kwargs,\
             arrow, train, val, test, data_set_name,\
             nf, gi, mode, T_idx, pruning, threshold, epo_idx,\
             batch_size, pre_train=pre_train, no_reg_epochs=no_reg_epochs, nf_epochs=epochs,\
             save_data=True, set_name=set_name, verbose=True)

lambda_nf5_1em1
start
Initilizing the model and optimizer
Size of the training array =  11.0 MB
N_training_samples = 179999, batch_size = 256, N_batch_updates_per_epoch = 703
Initial Validation sim-NRMS= 2.827166555436157


  1%|          | 1/100 [00:04<06:43,  4.07s/it]

########## New lowest validation loss achieved ########### sim-NRMS = 0.45637010865844774
Epoch    1, sqrt loss 0.09927, Val sim-NRMS 0.4564, Time Loss: 67.8%, data: 2.3%, val: 29.6%, 172.4 batches/sec


  2%|▏         | 2/100 [00:08<06:45,  4.14s/it]

########## New lowest validation loss achieved ########### sim-NRMS = 0.3257848213102114
Epoch    2, sqrt loss 0.02667, Val sim-NRMS 0.3258, Time Loss: 67.3%, data: 2.2%, val: 30.2%, 170.2 batches/sec


...

 98%|█████████▊| 49/50 [11:00<00:13, 13.41s/it]

Epoch   49, sqrt loss 0.01728, Val sim-NRMS 0.1058, Time Loss: 89.9%, data: 0.7%, val: 9.2%, 52.2 batches/sec


100%|██████████| 50/50 [11:13<00:00, 13.46s/it]

Epoch   50, sqrt loss 0.01727, Val sim-NRMS 0.1056, Time Loss: 89.9%, data: 0.7%, val: 9.2%, 52.2 batches/sec


arrow test complete!
test test complete!
arrow test complete!
test test complete!
done
